In [1]:
import os
import openai
openai.api_key = 'sk-xxx'

In [34]:
os.getenv("OPENAI_API_KEY")

# answer processing

In [2]:
import pandas as pd
llm_results = pd.read_csv('dataset/output/results_with_llm.csv')

In [19]:
llm_results.head(1)

,一级分类,二级分类,domain,数据来源,题型分类,prompt,answer,id,instruction,chatglm,chatglm2,chinese-alpaca-13b,moss-sft,baichuan-7b,davinci-003
0,基础能力,生成与创作,商品文案描述,机器生成/人工调整,问答,用以下关键词，帮我生成一段文案，关键词：牙刷架，家居，造型，小巧，收纳，小巧造型，牙刷，杯子,当家居遇上牙刷架，小巧造型的设计让收纳变得更加简单。这款牙刷架不仅外形精致小巧，还能同时容纳...,0,请你回答一个开放性问题，问题如下：\n用以下关键词，帮我生成一段文案，关键词：牙刷架，家居，...,"牙刷架是家居生活中必不可少的小物件之一。小巧玲珑的牙刷架不仅可以方便我们收纳牙刷,而且还可以...","\n欢迎来到我们的全新牙刷架——小巧、造型独特,为您带来全新的口腔护理体验!这款牙刷架采用小...",我们为您推荐一款小巧的牙刷架，它不仅可以帮助您更好地管理您的牙刷和杯子，还可以让您的家居更...,\n描述一种新型的小型电动牙刷架收纳盒。它采用轻巧的金属材质制作而成，外形设计简约大方、线条...,"1. 20世纪80年代,美国人发明了第一支电动牙刷。此后,随着人们对口腔卫生的重视程度不断...",如果你正在寻找一款小巧造型的牙刷架，那么它将是你家居收纳的完美选择。这款牙刷架不仅小巧，而且...


In [3]:
from src.llm_answer_postprocessor import AnswerProcessor
answer_processor = AnswerProcessor()

### example

In [17]:
d = dict(llm_results.iloc[4,:])
print(d['moss-sft'])
print('-'*100)
print(answer_processor.processing(typ=d['题型分类'], model_name='moss-sft', answer=d['moss-sft']))

<|Human|>: 我应该怎样安排饮食来减少体重？ <eoh> 
<|MOSS|>: 为了减轻体重并达到每周减重5磅的目标，您需要遵循以下建议的饮食计划: 

1. 增加蛋白质的摄入量。选择瘦肉、鱼和豆类等高质量蛋白质食物可以帮助你保持饱腹感并提供能量支持你的日常活动水平。每天摄入约100克左右的蛋白质是一个好的起点。
2. 降低碳水化合物的摄入量。尽量减少糖类和淀粉类食物的摄取，如糖果、蛋糕、面包等等。取而代之的是增加蔬菜、水果和低脂肪乳制品等的比例。这些食品可以提供足够的营养并且有助于控制食欲。
3. 多吃蔬菜和水果。它们富含纤维和其他营养素，而且热量较低。尝试每天吃至少4份不同颜色的蔬果以获取更多的营养成分。此外，将果汁替换为水或茶也可以帮助你控制添加的糖分含量。
4. 规律进餐并保持适当的饮水量。不要跳餐或者过度进食以避免暴饮暴食的情况发生。同时要保持足够的水分摄入以满足身体的需求。

总之，通过合理的饮食习惯和控制卡路里的消耗是实现减脂目标的必要条件之一。 <eom>
----------------------------------------------------------------------------------------------------
为了减轻体重并达到每周减重5磅的目标，您需要遵循以下建议的饮食计划: 

1. 增加蛋白质的摄入量。选择瘦肉、鱼和豆类等高质量蛋白质食物可以帮助你保持饱腹感并提供能量支持你的日常活动水平。每天摄入约100克左右的蛋白质是一个好的起点。
2. 降低碳水化合物的摄入量。尽量减少糖类和淀粉类食物的摄取，如糖果、蛋糕、面包等等。取而代之的是增加蔬菜、水果和低脂肪乳制品等的比例。这些食品可以提供足够的营养并且有助于控制食欲。
3. 多吃蔬菜和水果。它们富含纤维和其他营养素，而且热量较低。尝试每天吃至少4份不同颜色的蔬果以获取更多的营养成分。此外，将果汁替换为水或茶也可以帮助你控制添加的糖分含量。
4. 规律进餐并保持适当的饮水量。不要跳餐或者过度进食以避免暴饮暴食的情况发生。同时要保持足够的水分摄入以满足身体的需求。

总之，通过合理的饮食习惯和控制卡路里的消耗是实现减脂目标的必要条件之一。


# ChatGPT eval-agent

In [5]:
from src.eval_agent import ScoringAgent,UnderstandingAgent,RankingAgent
from src.llm_answer_postprocessor import AnswerProcessor
answer_processor = AnswerProcessor()
model_name = ['chatglm','chatglm2','chinese-alpaca-13b','moss-sft','baichuan-7b', 'davinci-003']
reference = 'answer'

## Scoring Example

In [8]:
example = dict(llm_results.iloc[5,:])

In [9]:
llm_answer = answer_processor.processing(typ=example['题型分类'], model_name='davinci-003', answer=example['davinci-003'])
inputs = {
    'question':example['prompt'],
    'llm_answer':llm_answer,
    'reference_answer':example['answer']
}
scoringAgent = ScoringAgent()
print(scoringAgent.get_prompt(**inputs))
print(scoringAgent.evaluation(inputs=inputs, 
                        model_name='gpt-3.5-turbo-16k-0613',
                        key=openai.api_key))


You are a scoring GPT, an intelligent assistant that can score the LLM output answer to the question with respect to following three aspects with score 1 to 3.

Aspect 1 effectiveness:
Score 1 means the answer is totally irrelevant to the question;
Score 2 means the answer can partially follow the question and provide insufficient information to the question;
Score 3 means the answer can follow the question well and provide enough information to the question.

Aspect 2 reliability:
Score 1 means the answer is totally fictitious and illogical;
Score 2 means the answer contains some facts faults and full of logical flaws;
Score 3 means the answer is logically sound and totally factual.

Aspect 3 fluency:
Score 1 means the answer is incomplete and incoherent, or keeps repeating the same phrase;
Score 2 means the answer is complete, while containing grammatical errors;
Score 3 means the answer is complete, coherent and free of grammatical errors.

Please output the score and the output fo

## Ranking Example

In [10]:
# dimension means evaluate the answer ranking from three dimension
ranking_agent = RankingAgent(method='dimension')
inputs = {
    'question':example['prompt'],
    'llm_answer':[answer_processor.processing(typ=example['题型分类'], model_name=a, answer=example[a]) for a in model_name],
    'reference_answer':example['answer']
}

print(ranking_agent.get_prompt(**inputs))
print(ranking_agent.evaluation(inputs=inputs, 
                        model_name='gpt-3.5-turbo-16k-0613',
                        key=openai.api_key))


This is RankGPT, an intelligent assistant that can rank answers based on their effectiveness, reliability, and fluency to the question. Definition of three principles are given below:
The definition of effectiveness are:
1. The answer can follow the question well and be related to the question;
2. The answer can provide some information to meet the requirements of the question.
The definition of reliability are:
1. The information in the answer is consistent with objective reality;
2. The answer and its reasoning are logically sound.
The definition of fluency are:
1. The answer is complete;
2. The answer is coherent and free of grammatical errors.

[1] 制定两个月的雅思备考计划,可以参考下述步骤和建议:
1. 评估英语水平:包括口语、写作、听力和阅读等方面。了解弱点和需要提高的领域,为接下来的备考计划做好准备。
2. 制定备考时间表:根据弱点和目标,制定一份详细的备考时间表。确保每天都有足够的时间来复习和学习。考虑到个人情况,可以安排一些时间来模拟考试和提高语法和词汇水平。
3. 学习雅思考试内容:在制定备考计划时,需要了解雅思考试的内容和要求。然后,针对每个部分进行深入学习和练习。可以参考官方的指南和练习题,例如 IELTS Practice Tests and Solutions。
4. 参加培训课程:如果还没有参加过雅思培训课程,可以考虑参加。这将帮助了解考试要求和测试方式,并提供实用的技巧和策略。
5. 练习

## Objective Question Understanding Example

In [6]:
understanding_agent = UnderstandingAgent()

In [9]:
from llm_inference import get_prompt_by_typ

prompt = get_prompt_by_typ(example)
llm_answer = answer_processor.processing(typ=example['题型分类'], model_name='davinci-003', answer=example['davinci-003'])
inputs = {
    'question':prompt,
    'llm_answer':llm_answer,
    'typ':'judgement'
}
print(understanding_agent.get_prompt(**inputs))
print(understanding_agent.evaluation(inputs=inputs, 
                        model_name='gpt-3.5-turbo-16k-0613',
                        key=openai.api_key))


Following is a judgement question and the corresponding answer, please tell me the attitude of answer to the question.
Here are three options for you to choose:
A. The answer totally thinks the argument in the question is correct
B. The answer partially agree or disagree with the argument in the question
C. The answer is irrelevant with the argument in the question, or does not answer the question

Question:请你回答一个开放性问题，问题如下：
帮我制定两个月的雅思备考计划，并推荐学习资料  现在，请开始回答：

Answer:在两个月的雅思备考计划中，我建议您每周至少安排4-5个小时的学习时间，每天安排1-2个小时的学习时间，每天安排一次模拟考试，以检验自己的学习效果。在学习过程中，要重点掌握雅思考试的考试形式、考试要求、考试技巧等，并熟练掌握各种语言知识，如语法、词汇、句型等。

推荐学习资料方面，我推荐您可以使用官方指定的雅思考试教材，如《Official IELTS Practice Materials》、《Official IELTS Practice Materials 2》等，这些教材可以帮助您更好地掌握雅思考试的知识点，并且可以提供大量的模拟试题，以便您更好地检验自己的学习效果。此外，您还可以使用一些网络资源，如慕课网、沪江网等，这些网络资源可以提供大量的学习资料，以及一些有用的学习技巧，可以帮助您更好地备考雅思考试。
Please output A/B/C only after the symbol [Answer]. 
For example:
[Answer] A

The attitude of the answer is:
[Answer]

B
